In [54]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import anthropic
import gradio as gr

/Users/saud/Desktop/LLM_Projects/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key and openai_api_key.startswith('sk-proj-'):
    print('API key exists, good to go.')
else:
    print("API key doesn't exist, troubleshoot the problem")

API key exists, good to go.


In [49]:
openai = OpenAI()
GPT_MODEL = 'gpt-4o-mini'
anthropic = anthropic.Anthropic()
CLAUDE_MODEL = 'claude-3-5-sonnet-20240620'

system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. "
system_message += "#include all necesarry libaries or packages that are required for a C++ code."


def user_message(python_code):
    prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    prompt += python_code
    return prompt

In [6]:
def message_for(python_code):
    return [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message(python_code)}
    ]
    

In [38]:
def write_cpp_code_to_file(cpp_code):
    code = cpp_code.replace('```cpp', "").replace('```', "")
    with open('optimized.cpp', 'w') as f:
        f.write(code)
    

In [65]:
def stream_gpt_response(python_code):
    response = openai.chat.completions.create(
        model= GPT_MODEL,
        messages= message_for(python_code),
        stream= True
    )
    result = ""
    for chunk in response:
        stream = chunk.choices[0].delta.content or ""
        result += stream
    yield result.replace('```cpp', '').replace('```', '')
    
    write_cpp_code_to_file(result)


In [66]:
def stream_claude_response(python_code):
    response = anthropic.messages.stream(
        model= CLAUDE_MODEL,
        system= system_message,
        messages= [{"role": "user", "content": user_message(python_code)}],
        max_tokens= 200
    )
    result = ""
    with response as stream:
        for chunk in stream.text_stream:
            result += chunk
    yield result.replace('```cpp', '').replace('```', '')
    
    write_cpp_code_to_file(result)


In [67]:
python = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [61]:
exec(python)

Result: 3.141592658589
Execution Time: 21.394866 seconds


In [62]:
stream_gpt_response(python)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int64_t iterations, double param1, double param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4.0, 1.0) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> elapsed = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << elapsed.count() << " seconds" << std::endl;

    return 0;
}
```

In [68]:
def select_streaming_model(model, python_code):
    if model == 'GPT':
        response = stream_gpt_response(python_code)
    elif model == 'Claude':
        response = stream_claude_response(python_code)
    else:
        raise ValueError("Unknown model")

    for streaming_text in response:
        yield streaming_text

In [ ]:
def select_streaming_model(model, python_code):
    if model == 'GPT':
        response = stream_gpt_response(python_code)
    elif model == 'Claude':
        response = stream_claude_response(python_code)
    else:
        raise ValueError(f"Unknown model: {model}")

    for streaming_text in response:
        yield streaming_text

with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    # Fix the order of inputs
    convert.click(select_streaming_model, inputs=[model, python], outputs=[cpp])

ui.launch(inbrowser=True)


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
